In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf

tf.get_logger().setLevel("ERROR")
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp


# Using Bicubic Downscaling

In [ ]:
scale_factor = 4
resize_method = tf.image.ResizeMethod.BICUBIC

builder_kwargs = {
    "resize_method": resize_method,
    "scale": scale_factor,
    "antialias": True,
}
images = (
    tfds.load(name="set14", split="test", builder_kwargs=builder_kwargs)
    .map(datasets.map_on_dict(datasets.to_float32))
    .map(datasets.map_on_dict(datasets.from_255_to_1_range))
)


In [ ]:
image_dict = datasets.get_one_example(images, index=2)
image_hr = image_dict["hr"][None, ...]
image_lr = image_dict["lr"][None, ...]


### Reconstruct

In [ ]:
# Load the denoiser
denoiser, (denoiser_min_stddev, denoiser_max_stddev) = dppp.load_denoiser(
    "models/drugan+_0.0-0.2.h5"
)

# Define the noise stddev used for the stochastic evaluation of the prior
if denoiser_min_stddev <= 0.1 <= denoiser_max_stddev:
    denoiser_stddev = 0.1
else:
    denoiser_stddev = denoiser_max_stddev

# Callbacks: Print the PSNR every 20th step
callbacks = [dppp.callback_print_psnr("psnr", 20, image_hr)]

# Reconstruct the image
reconstructed = dppp.dmsp_super_resolve(
    degraded=image_lr,
    sr_factor=scale_factor,
    denoiser=denoiser,
    denoiser_stddev=denoiser_stddev,
    num_steps=300,
    callbacks=callbacks,
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(image_hr, reconstructed).numpy()[0]
lpips = dppp.lpips_alex(image_hr, reconstructed).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([image_hr[0], image_lr[0], reconstructed[0]])

# Using a Gaussian Blur Kernel

In [ ]:
scale_factor = 2
images = (
    tfds.load(name="cbsd68", split="test")
    .map(datasets.get_image)
    .map(datasets.to_float32)
    .map(datasets.from_255_to_1_range)
)
image_hr = datasets.get_one_example(images, index=31)[None, :480, :320, :]
kernel = dppp.conv2D_filter_rgb(dppp.ZHANG_GAUSSIAN_KERNELS[6])
image_blurred = dppp.blur(image_hr, kernel, 0, mode="wrap")
image_lr = image_blurred[:,::scale_factor,::scale_factor,:]

### Reconstruct

In [ ]:
# Load the denoiser
denoiser, (denoiser_min_stddev, denoiser_max_stddev) = dppp.load_denoiser(
    "models/drugan+_0.0-0.2.h5"
)

# Define the noise stddev used for the stochastic evaluation of the prior
if denoiser_min_stddev <= 0.1 <= denoiser_max_stddev:
    denoiser_stddev = 0.1
else:
    denoiser_stddev = denoiser_max_stddev

# Callbacks: Print the PSNR every 20th step
callbacks = [dppp.callback_print_psnr("psnr", 20, image_hr)]

reconstructed = dppp.dmsp_super_resolve(
    degraded=image_lr,
    sr_factor=scale_factor,
    denoiser=denoiser,
    denoiser_stddev=denoiser_stddev,
    kernel=kernel,
    callbacks=callbacks,
    conv_mode="wrap",
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(image_hr, reconstructed).numpy()[0]
lpips = dppp.lpips_alex(image_hr, reconstructed).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([image_hr[0], image_lr[0], reconstructed[0]])